### SST OM4p25

In [1]:
import cartopy
import cmocean
import matplotlib.pyplot as plt
import matplotlib.colors
import matplotlib.colorbar
import netCDF4
import numpy

In [2]:
%%time
## net series files 300yr OM4p25
OM4p25 = netCDF4.MFDataset('/work/Nkeh.Boh/SST/OM4p25-100yr/Netfile/*.nc') #100 yr

W25 = netCDF4.Dataset('/work/Nkeh.Boh/SST/NetOMfiles/W25/WOA05_ptemp_salt_monthly.v20141007.nc')

W25_av  = numpy.mean(W25['ptemp'][:],axis=(0,1))
OM4p25_av = numpy.mean(OM4p25['tos'][:],axis=(0,1))

#anomaly = OM4p25_av - W25_av


CPU times: user 19.1 s, sys: 29.8 s, total: 48.9 s
Wall time: 48.9 s


# Plots

#### Doing typical time averaging + plotting

In [ ]:
%%time
G25 = netCDF4.Dataset('/archive/gold/datasets/OM4_025/mosaic.v20170622.unpacked/ocean_hgrid.nc')
G5 = netCDF4.Dataset('/archive/gold/datasets/OM4_05/mosaic_ocean.v20180227.unpacked/ocean_hgrid.nc')
M25 = netCDF4.Dataset('/archive/gold/datasets/OM4_025/mosaic.v20170622.unpacked/ocean_mask.nc')
M5 = netCDF4.Dataset('/archive/gold/datasets/OM4_05/mosaic_ocean.v20180227.unpacked/ocean_mask.nc')

xq25 = G25.variables['x'][:][::2,::2]
yq25 = G25.variables['y'][:][::2,::2]
a25 = G25.variables['area'][:]; a25 = a25[::2,::2]+a25[1::2,1::2]+a25[1::2,::2]+a25[1::2,::2]
m25 = M25.variables['mask'][:]; a25 = a25*m25


def stats(ax, area, anomaly, label):
    mn = (anomaly*area).sum()/area.sum()
    sd = numpy.sqrt( ((anomaly-mn)**2*area).sum()/area.sum() )
    rms = numpy.sqrt( (anomaly**2*area).sum()/area.sum() )
    qmn, qmx = anomaly.min(), anomaly.max()
    print(label, 'mean =', mn, 'sd =', sd, 'rms =', rms, 'min =', qmn, 'max =', qmx )
    bb = ax.get_position()
    plt.gcf().text(bb.x0,bb.y1+.01,'mean=%.3f$^\circ$C'%mn, horizontalalignment='left')
    plt.gcf().text(bb.x1,bb.y1+.01,'rms=%.3f$^\circ$C'%rms, horizontalalignment='right')


fig = plt.figure(figsize=(10,7))
vmin,vmax,ci,cmap = -2.25,2.25,.5,plt.cm.RdYlBu_r
# axes = []
cilev = numpy.arange(vmin-ci,vmax+ci*2,ci)
norm = matplotlib.colors.BoundaryNorm(boundaries=cilev, ncolors=cmap.N)

# anomaly = OM4p25_av - W25_av

ax = plt.axes(projection=cartopy.crs.Robinson(central_longitude=-155))

im = ax.pcolormesh(xq25, yq25, anomaly, transform=cartopy.crs.PlateCarree(), cmap=cmap, norm=norm, vmin=vmin, vmax=vmax)
stats(ax, a25, anomaly, 'OMp25')
ax.coastlines()
# axes.append(ax)
plt.title('a) OM4p25')

ax = plt.gcf().add_axes((.25,.5,.5,.03))
cb = matplotlib.colorbar.ColorbarBase(ax=ax, cmap=cmap, norm=norm, boundaries=cilev,
                                      orientation='horizontal', extend='both')
cb.ax.set_title('SST bias ($^\circ$C)')